In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
pd.set_option('display.max_columns', None)

In [2]:
dataPath = "data/"

pairs = pd.read_csv(dataPath + "relation.csv")
users = pd.read_csv(dataPath + "customers.csv", index_col="id")
items = pd.read_csv(dataPath + "products.csv", index_col="id")


/tmp/ipykernel_1118/1774659839.py:4: DtypeWarning: Columns (7,9,10,11,12,13,17,18,21,26,27,28,30,36) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv(dataPath + "customers.csv", index_col="id")
/tmp/ipykernel_1118/1774659839.py:5: DtypeWarning: Columns (20,32,34,39) have mixed types. Specify dtype option on import or set low_memory=False.
  items = pd.read_csv(dataPath + "products.csv", index_col="id")


### Compañia 15

In [3]:
df_pairs = pairs[ pairs.company_id == 15]#.copy()
df_users = users[ users.company_id == 15]#.copy()
df_items = items[ items.company_id == 15]#.copy()

In [4]:
# Comprobacion de que no haya usuarios ni productos duplicados

assert df_users.index.duplicated().any()==False
assert df_items.index.duplicated().any()==False

### Quitar usuarios que interaccionan pero no estan contemplados en la tabla usuarios

In [5]:
usuarios_que_interaccionan_pero_no_en_tabla_usuarios = set(df_pairs.customer_id) - set(df_users.index)
len(usuarios_que_interaccionan_pero_no_en_tabla_usuarios)

575

In [6]:
df_pairs = df_pairs[ ~df_pairs.customer_id.isin(usuarios_que_interaccionan_pero_no_en_tabla_usuarios) ]
len(df_pairs)

59167

### Quitar productos que interaccionan pero no estan contemplados en la tabla productos

In [7]:
productos_que_interaccionan_pero_no_en_tabla_productos = set(df_pairs.product_id) - set(df_items.index)
len(productos_que_interaccionan_pero_no_en_tabla_productos)

162

In [8]:
df_pairs = df_pairs[ ~df_pairs.product_id.isin(productos_que_interaccionan_pero_no_en_tabla_productos) ]
len(df_pairs)

58913

# Contar usuarios y priductos

In [9]:
ACTIVE_USERS = df_pairs.customer_id.unique()
ACTIVE_ITEMS = df_pairs.product_id.unique()

real2fake_userId = {userId:i for i,userId in enumerate(ACTIVE_USERS)}
real2fake_itemId = {itemId:i for i,itemId in enumerate(ACTIVE_ITEMS)}

NUM_ACTIVE_USERS = len(ACTIVE_USERS)
NUM_ACTIVE_ITEMS = len(ACTIVE_ITEMS)

NUM_USERS = len(df_users)
NUM_ITEMS = len(df_items)

print(f"Hy un total de {NUM_ACTIVE_USERS} usuarios que interaccionan de un total de {NUM_USERS} usuarios")
print(f"Hy un total de {NUM_ACTIVE_ITEMS} prudcutos que interaccionan de un total de {NUM_ITEMS} productos")

Hy un total de 28806 usuarios que interaccionan de un total de 54029 usuarios
Hy un total de 477 prudcutos que interaccionan de un total de 780 productos


## Cambiar ids

In [10]:
df_pairs.sample(2)

,company_id,product_id,customer_id,compras,visto,peso,ultimo
62870,15,28931,51745,1,0,3.0,2020-12-27 01:26:34
84846,15,29124,24649,1,0,3.0,2017-05-31 14:01:22


In [ ]:
df_users_feats

In [36]:
df_items_feats = pd.concat(
    [
        (( df_items.price - df_items.price.mean() ) / df_items.price.std() ).to_frame(),
        df_items.features.str.get_dummies(','),
        df_items.families.str.get_dummies(','),
    ],
    axis="columns"    
)
df_items_feats

price  10040  10185  10191  10206  10239  10289  10306  10346  \
id                                                                         
28863  -0.172383      0      0      0      0      0      0      0      0   
28864   0.009349      0      0      0      0      0      0      0      0   
28865   0.099989      0      0      0      0      0      0      0      0   
28866   0.235948      0      0      0      0      0      0      0      0   
28867   0.598506      0      0      0      0      0      0      0      0   
...          ...    ...    ...    ...    ...    ...    ...    ...    ...   
126891 -0.624675      0      0      0      0      0      0      0      0   
126951 -0.624675      0      0      0      0      0      0      0      0   
126952 -0.624675      0      0      0      0      0      0      0      0   
126957 -0.624675      0      0      0      0      0      0      0      0   
126958 -0.624675      0      0      0      0      0      0      0      0   

        10362  10433  10513  10537  10582  10588  10899  10926  10960  11035  \
id                                                                             
28863       0      0      0      0      0      0      0      0      0      0   
28864       0      0      0      0      0      0      0      0      0      0   
28865       0      0      0      0      0      0      0      0      0      0   
28866       0      0      0      0      0      0      0      0      0      0   
28867       0      0      0      0      0      0      0      0      0      0   
...       ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
126891      0      0      0      0      0      0      0      0      0      0   
126951      0      0      0      0      0      0      0      0      0      0   
126952      0      0      0      0      0      0      0      0      0      0   
126957      0      0      0      0      0      0      0      0      0      0   
126958      0      0      0      0      0      0      0      0      0      0   

        11044  11066  1152  11606  1163  11806  11880  11881  11890  11899  \
id                                                                           
28863       0      0     0      0     0      0      1      0      0      0   
28864       0      0     0      0     0      0      1      0      0      0   
28865       0      0     0      0     0      0      1      0      0      0   
28866       0      0     0      0     0      0      1      0      0      0   
28867       0      0     0      0     0      0      1      0      0      0   
...       ...    ...   ...    ...   ...    ...    ...    ...    ...    ...   
126891      0      0     0      0     0      0      0      0      0      0   
126951      0      0     0      0     0      0      0      0      0      0   
126952      0      0     0      0     0      0      0      0      0      0   
126957      0      0     0      0     0      0      0      0      0      0   
126958      0      0     0      0     0      0      0      0      0      0   

        1210  1230  12693  12929  1308  13130  13735  14057  14093  14242  \
id                                                                          
28863      0     0      0      0     0      0      0      0      0      0   
28864      0     0      0      0     0      0      0      0      0      0   
28865      0     0      0      0     0      0      0      0      0      0   
28866      0     0      0      0     0      0      0      0      0      0   
28867      0     0      0      0     0      0      0      0      0      0   
...      ...   ...    ...    ...   ...    ...    ...    ...    ...    ...   
126891     0     0      0      0     0      0      0      0      0      0   
126951     0     0      0      0     0      0      0      0      0      0   
126952     0     0      0      0     0      0      0      0      0      0   
126957     0     0      0      0     0      0      0      0      0      0   
126958     0     0      0      0     0    

In [16]:
class CollabFilteringDataset(torch.utils.data.Dataset):
    
    def __init__(self, df_pairs, df_users, df_items):
        
        self.df_pairs = df_pairs
        self.df_pairs["fake_user_id"] = self.df_pairs["customer_id"].map(real2fake_userId)
        self.df_pairs["fake_item_id"] = self.df_pairs["product_id"].map(real2fake_itemId)
        
        # Extract features: item_id + 1 + 37 + 1027
        self.item_id       = df_items.index  # Categorica, embedding
        self.item_prices   = ( df_items.price - df_items.price.mean() ) / df_items.price.std() # Continua (1)
        self.item_families = df_items.families.str.get_dummies(',') # Categorica, N-hot encoding (37)
        self.item_features = df_items.features.str.get_dummies(',') # Categorica, N-hot encoding (1027)
        
        self.user_id     = df_users.index # Categorica, embedding
        self.user_gender = pd.get_dummies(df_users.gender_id, dummy_na = True) # Categorica, 1-hot encoding (4)

    def __len__(self):
        return len(self.df_pairs)

    def __getitem__(self, idx):
        
        pairs_row = self.df_pairs.iloc[idx]
        
        item_fake_id  = pairs_row.fake_item_id # Para usarlo en el embedding
        item_id       = pairs_row.product_id   # ... para obetener mas datos del item (clave ajena)
        item_price    = self.item_prices.loc[item_id]
        item_families = self.item_families.loc[item_id].values
        item_features = self.item_features.loc[item_id].values
        
        user_fake_id  = pairs_row.fake_item_id # Para usarlo en el embedding
        user_id       = pairs_row.customer_id  # ...para obetener mas datos del user (clave ajena)
        user_gender   = self.user_gender.loc[user_id].values
        
        peso          = pairs_row.peso
                
        return {"Item_id": item_fake_id, 
                "User_id": user_fake_id,
                "Item_feats": np.concatenate((np.array([item_price], dtype="float32"),
                                              item_families.astype("float32"),
                                              item_features.astype("float32") )),
                "User_feats": user_gender.astype("float32"),
                "Peso": peso}

In [17]:
ds = CollabFilteringDataset(df_pairs, df_users, df_items)

In [18]:
ds[0]

{'Item_id': 0,
 'User_id': 0,
 'Item_feats': array([-0.17238298,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ], dtype=float32),
 'User_feats': array([1., 0., 0., 0.], dtype=float32),
 'Peso': 3.0}

In [43]:
train_size = int(0.8 * len(ds))
valid_size = len(ds) - train_size
train_ds, valid_ds = torch.utils.data.random_split(ds, [train_size, valid_size])

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=64, shuffle=False)

## Model

In [48]:
class Model(torch.nn.Module):
    
    def __init__(self): # n_users, n_movies, n_factors, y_range=(0,5.5)
        
        super().__init__()
        
        COMMON_EMB_DIM = 32
        
        self.user_id_embed_W = torch.nn.Embedding(NUM_ACTIVE_USERS, COMMON_EMB_DIM)
        self.user_id_embed_B = torch.nn.Embedding(NUM_ACTIVE_USERS, 1)
        self.user_linear_1   = torch.nn.Linear(4, COMMON_EMB_DIM) # 4 (genero)
                
        self.item_id_embed_W = torch.nn.Embedding(NUM_ACTIVE_ITEMS, COMMON_EMB_DIM)
        self.item_id_embed_B = torch.nn.Embedding(NUM_ACTIVE_ITEMS, 1)
        self.item_linear_1   = torch.nn.Linear(1065, 200) # 1 (precio) + 37 (familias) + 1027 (features)
        self.item_linear_2   = torch.nn.Linear(200, COMMON_EMB_DIM)
        
        """
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias    = Embedding(n_users, 1)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_movies, 1)
        self.y_range = y_range
        """;
        
    def encode_user(self, real_user_id, user_feats):        
        user_emb_from_id    = self.user_id_embed_W(user_id)
        user_emb_from_feats = self.user_linear_1(user_feats)
        
        return user_emb_from_feats + user_emb_from_id
    
    
    def encode_active_user(self, user_id, user_feats):        
        user_emb_from_id    = self.user_id_embed_W(user_id)
        user_emb_from_feats = self.user_linear_1(user_feats)
        
        return user_emb_from_feats + user_emb_from_id
        
        
    def encode_active_item(self,  item_id, item_feats):
        item_emb_from_id    = self.item_id_embed_W(item_id)
        item_emb_from_feats = self.item_linear_1(item_feats)
        item_emb_from_feats = torch.nn.functional.relu(item_emb_from_feats)
        item_emb_from_feats = self.item_linear_2(item_emb_from_feats)
        
        return item_emb_from_feats + item_emb_from_id

    
    def forward(self, x):
        
        user_embedding = self.encode_active_user(x["User_id"], x["User_feats"])
        item_embedding = self.encode_active_item(x["Item_id"], x["Item_feats"])
        
        return (user_embedding * item_embedding).sum(axis=1)

        """
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
        return sigmoid_range(res, *self.y_range)
        """;

In [49]:
m = Model()

In [50]:

for batch in tqdm(train_dl):
    #print("========")
    out = m(batch)
    #print(out)


100%|█████████████████████████████████████████| 737/737 [00:09<00:00, 76.98it/s]


In [24]:
m 

Model(
  (user_id_embed_W): Embedding(28806, 32)
  (user_id_embed_B): Embedding(28806, 1)
  (user_linear_1): Linear(in_features=4, out_features=32, bias=True)
  (item_id_embed_W): Embedding(477, 32)
  (item_id_embed_B): Embedding(477, 1)
  (item_linear_1): Linear(in_features=1065, out_features=200, bias=True)
  (item_linear_2): Linear(in_features=200, out_features=32, bias=True)
)